In [2]:
%matplotlib inline

import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from pandas.tools.plotting import table
import json
from IPython.display import display
import re
import string


In [3]:
#Load in file

#with open('J1553+3548_IGM.json') as data_file:    
    #data = json.load(data_file)
    
with open('dusty3_J1016+4706.json') as data_file:    
    data = json.load(data_file)

In [20]:
#Read in data from file, identify systems and components

systems=['z0.00000_MW']
components=[]


for cmp in data["cmps"]:
    cmp_dict=data["cmps"][str(cmp)]
    cmp_data=''
    if cmp[-2:]=="HI":
        systems.append(str(cmp))
    cmp_data+=str(cmp)+';'
    cmp_data+=str(cmp_dict["Reliability"])+';'
    if cmp_dict["Comment"]=='':
        cmp_data+='None;'
    else:
        cmp_data+=str(cmp_dict["Comment"])+';'
    cmp_data+=str(cmp_dict["Nfit"])+';'
    cmp_data+=str(cmp_dict["bfit"])+';'
    components.append(cmp_data)

In [21]:
def letters(input):
    valids = []
    for character in input:
        if character.isalpha():
            valids.append(character)
    return ''.join(valids)

In [6]:
#Sort systems by redshift

def atoi(text):
    return int(text) if text.isdigit() else text

def natural_keys(text):
    return [ atoi(c) for c in re.split('(\d+)', text) ]


systems.sort(key=natural_keys)
print(systems)

#for system in systems[1:]:
    #for line in data["cmps"][system]["lines"].keys():
        #print line

['z0.00000_MW', 'z0.00361_HI', 'z0.00668_HI', 'z0.03132_HI', 'z0.03554_HI', 'z0.03604_HI', 'z0.03859_HI', 'z0.04119_HI', 'z0.04145_HI', 'z0.04432_HI', 'z0.09876_HI', 'z0.11520_HI', 'z0.14261_HI', 'z0.14751_HI', 'z0.14915_HI', 'z0.16589_HI', 'z0.17067_HI', 'z0.17487_HI', 'z0.17621_HI', 'z0.17947_HI', 'z0.18067_HI', 'z0.18138_HI', 'z0.19132_HI', 'z0.20026_HI', 'z0.21968_HI', 'z0.22011_HI', 'z0.22063_HI', 'z0.24294_HI', 'z0.25215_HI', 'z0.25281_HI', 'z0.26330_HI', 'z0.26378_HI', 'z0.26412_HI', 'z0.27319_HI', 'z0.27884_HI', 'z0.28019_HI', 'z0.29016_HI', 'z0.29058_HI', 'z0.29307_HI', 'z0.29402_HI', 'z0.32906_HI', 'z0.35225_HI', 'z0.35598_HI', 'z0.35972_HI', 'z0.36794_HI', 'z0.38571_HI', 'z0.39994_HI', 'z0.40308_HI', 'z0.40871_HI', 'z0.41027_HI', 'z0.42938_HI', 'z0.43214_HI', 'z0.46139_HI', 'z0.66463_HI', 'z0.72770_HI', 'z0.74627_HI']


In [22]:
H_z = []
Element = []

for sys in systems[1:]:
    sys = sys.replace('z', '')
    Element.append(letters(sys))
    sys = sys.replace('_HI','')
    z_shift = float(sys)
    H_z.append(z_shift)

print Element
print H_z

['HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI', 'HI']
[0.7277, 0.2633, 0.17487, 0.04145, 0.22011, 0.40308, 0.26412, 0.1152, 0.40871, 0.32906, 0.22063, 0.18067, 0.18138, 0.14261, 0.46139, 0.03554, 0.24294, 0.66463, 0.25281, 0.04432, 0.27884, 0.04119, 0.17067, 0.29307, 0.09876, 0.17621, 0.43214, 0.26378, 0.14751, 0.27319, 0.17947, 0.35598, 0.00361, 0.36794, 0.29058, 0.00668, 0.16589, 0.03859, 0.41027, 0.42938, 0.35225, 0.20026, 0.74627, 0.28019, 0.29016, 0.25215, 0.21968, 0.38571, 0.14915, 0.29402, 0.19132, 0.03132, 0.35972, 0.03604, 0.39994]


In [29]:
#Sort components by system and visualize

#Note: Systems that are close together (<~0.010) may overlap components in the current build.

Sorted_sys = {}
col_den = []
for system in H_z:
    sys_components=[]
    sys_reliabilities=[]
    sys_comments=[]
    sys_n=[]
    sys_b=[]
    element = []
    z_shift = []
    column = []
    H_systems = {}
    sys_dict={}
    count = 0
    total_cz = []
    for component in components:
        #print component
        if component[1]=='-':
            c_z=component[2:9]
        else:
            c_z=component[1:8]
        total_cz.append(float(c_z))
        cmp_info = component.split(';')
        column.append(float(cmp_info[3]))
        
        if abs(float(c_z)-system)<=0.00167: ## 500 km/s offset. 
            cmp_info=component.split(';')
            element.append(letters(cmp_info[0][1:]))
            z_shift.append(float(cmp_info[0][1:7]))                
            sys_components.append(cmp_info[0])
            sys_reliabilities.append(cmp_info[1])
            sys_comments.append(cmp_info[2])
            sys_n.append(cmp_info[3])
            sys_b.append(cmp_info[4])
            sys_dict={'Component':sys_components,'Reliability':sys_reliabilities,'Comment': sys_comments, 'Column Density (N)': sys_n, 'Doppler Profile (b)':sys_b}
        #if ele == "HI":
         #   H_systems[z_shift] = sys_n
            #print H_systems    
    #print "==================================================="
    #print "System: "+system
    #print "==================================================="
    #df = pd.DataFrame(sys_dict,columns = ['Component','Column Density (N)','Doppler Profile (b)', 'Reliability', 'Comment'])
    #display(df)
            

In [8]:
print sys_dict

{'Comment': ['OIV', 'Lyman series'], 'Doppler Profile (b)': ['18.2', '19.3'], 'Reliability': ['a', 'a'], 'Component': ['z0.74624_OIV', 'z0.74627_HI'], 'Column Density (N)': ['14.04', '15.44']}


In [13]:
"""
Major Systems: z = [0.00668, 0.11520, 0.16586, 0.24294, 0.25215, 
0.25281, 0.26330, 0.26378, 0.26412, 0.29307, 0.38571, 0.43214, 
0.66463, 0.72770, 0.74627]

z = [0.271857, 0.116515, 0.215892, 0.083528, 0.236281, 0.102918,
    0.149886, 0.15304, 0.177254, 0.160121, 0.089705, 0.051373,
    0.128096]

galaxy[(154.091787, 17.112158),(154.094096, 47.1165247)
        (154.099184, 47.101724), (154.087127, 47.105872), 
        (154.119066, 47.113131), (154.091440, 47.094367)
        (154.07192, 47.140234 Spectrum), (154.110013, 47.147945 Spectrum)
        (154.020562, 47.146454 Spectrum) (154.029008, 47.136826 Spectrum)
        (154.110217, 47.06976 Spectrum), (153.790990, 46.99868 Spectrum)
        (154.224976, 46.991421 Spectrum)]
        
        
        ra=154.38140626, dec=46.98530098
        nnAvgZ	0.046061
        
        ra=153.94045797, dec=47.33027694
        nnAvgZ	0.079424
        
        """

'z = [0.26412,0.11520,0.21968]\ngalaxy[(154.091787, 17.112158),(154.094096, 47.1165247)\n        (154.099184, 47.101724), ()]'